#### Задание

Собрать информацию о вакансиях на вводимую должность с сайтa [hh.ru](https://hh.ru/?customDomain=1). Приложение должно анализировать несколько страниц сайта.
Получившийся список должен содержать в себе:
- Наименование вакансии
- Предлагаемую зарплату (дополнительно: разносим в три поля: минимальная, максимальная, валюта. Цифры преобразуем к цифрам)
- Ссылку на саму вакансию
- Сайт, откуда собрана вакансия

По желанию можно добавить еще параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для всех вакансий. Общий результат можно вывести с помощью dataFrame через pandas, сохранить в json, либо csv.

#### Решение

In [6]:
# импортируем необходимые библиотеки

import pandas as pd
import requests
import json
from bs4 import BeautifulSoup as bs

In [7]:
# задаем headers
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

In [8]:
# создаем наш итоговый список с вакансиями    
vacancy_list = []

# проходим циклом по 10 страницам
for i in range(10):
    
    # Задаем адреса страниц - у нулевой страницы адрес отличается
    if i == 0:
        url = 'https://hh.ru/vacancies/data-engineer?customDomain=1'
    else:
        url = f'https://hh.ru/vacancies/data-engineer?customDomain=1&page={i}&hhtmFrom=vacancy_search_catalog'

    # парсим страницу целиком
    response = requests.get(url, headers = headers)
    dom = bs(response.text, 'html.parser')

    # создаем список тегов <div> у которых class = 'vacancy-serp-item__layout', что соответсвует блоку одной вакансии (посмотрели в браузере)
    tag_div_list = dom.find_all('div', {'class' : ['vacancy-serp-item__layout']})

    # проходим циклом по нашему списку вакансий со страницы
    for tag_div in tag_div_list:

        # задаем словарь с информауией о вакансии
        tag_div_dict = {}

        # находим тег с вакансией и ссылкой
        find_vacancy = tag_div.find('a', {'class' : 'serp-item__title'})

        # находим значение зарплаты, через try/except, так как не всегда ЗП указана, в случае ошибки, присваиваем значение 'ЗП не указана'
        try:
            salary_vacancy = tag_div.find('span', {'data-qa' : 'vacancy-serp__vacancy-compensation'}).text.replace('\u202f', ' ')
        except AttributeError:
            salary_vacancy = 'ЗП не указана'

        # находим из find_vacation ссылку на вакансию
        href = find_vacancy['href']

        # создаем словарь где ключ - название вакансии, а значения - это ЗП и ссылка на вакансию.
        tag_div_dict[find_vacancy.text] = {'salary': salary_vacancy,'link': href, 'site': 'hh.ru'}

        # добавляем в наш итоговый список словарь с информацией о вакансии
        vacancy_list.append(tag_div_dict)


In [56]:
vacancy_list

[{'Data engineer': {'salary': 'ЗП не указана',
   'link': 'https://hh.ru/vacancy/73471184?query=data+engineer',
   'site': 'hh.ru'}},
 {'Python developer (Machine Learning Engineer)': {'salary': 'ЗП не указана',
   'link': 'https://hh.ru/vacancy/73468878?query=data+engineer',
   'site': 'hh.ru'}},
 {'Middle/Senior Data Engineer в Sber AI Lab': {'salary': 'ЗП не указана',
   'link': 'https://hh.ru/vacancy/73471713?query=data+engineer',
   'site': 'hh.ru'}},
 {'NLP исследователь в группу AI в медицине Sber AI Lab': {'salary': 'ЗП не указана',
   'link': 'https://hh.ru/vacancy/73469286?query=data+engineer',
   'site': 'hh.ru'}},
 {'Data engineer': {'salary': 'ЗП не указана',
   'link': 'https://hh.ru/vacancy/72030869?query=data+engineer',
   'site': 'hh.ru'}},
 {'Python developer (Machine Learning Engineer)': {'salary': 'ЗП не указана',
   'link': 'https://hh.ru/vacancy/72074865?query=data+engineer',
   'site': 'hh.ru'}},
 {'Middle/Senior Data Engineer в Sber AI Lab': {'salary': 'ЗП не ук

In [129]:
# Если хотим преобразовать в DataFrame, то изменим структуру в словарь

# создаем словарь, нужной структуры
dict_for_DF = {'vacancy' : [], 'salary' : [], 'link' : [], 'site': []}

# пробегаемся циклом по каждой вакансии из цикла
for vacancy in vacancy_list:
    
    # добавляем в список по ключу 'vacancy' значение - название вакансии
    dict_for_DF['vacancy'].append(*list(vacancy.keys()))
    
    # добавляем в списки по остальным ключам значения, исключая уже первый ключ 'vacancy'
    for col in list(dict_for_DF.keys())[1:]:
        dict_for_DF[col].append(list(vacancy.values())[0][col])

dict_for_DF

{'vacancy': ['Data engineer',
  'Python developer (Machine Learning Engineer)',
  'Middle/Senior Data Engineer в Sber AI Lab',
  'NLP исследователь в группу AI в медицине Sber AI Lab',
  'Data engineer',
  'Python developer (Machine Learning Engineer)',
  'Middle/Senior Data Engineer в Sber AI Lab',
  'Data Engineer (Senior/Middle+)',
  'Младший разработчик баз данных MS SQL/Data engineer',
  'Data Engineer',
  'Engineering manager (Serbia)',
  'Data Engineer DWH',
  'Senior Machine Learning Engineer',
  'Data Engineer',
  'Data Engineer',
  'Senior Data Engineer',
  'Data Engineer/Дата Инженер',
  'Junior Data Engineer / Data Analyst',
  'Data Engineer',
  'Data Scientist',
  'Data Engineer',
  'Дата-сайентист / Data Scientist (Computer Vision)',
  'Lead Data Engineer',
  'Data Scientist',
  'Data Scientist',
  'Data Engineer',
  'Data Engineer/Инженер данных',
  'Data Engineer',
  'Data engineer',
  'Data engineer /Разработчик dwh',
  'Data Engineer',
  'PBI Data Engineer',
  'Head D

In [131]:
# теперь преобразуем в DataFrame

df = pd.DataFrame(dict_for_DF)
df

,vacancy,salary,link,site
0,Data engineer,ЗП не указана,https://hh.ru/vacancy/73471184?query=data+engi...,hh.ru
1,Python developer (Machine Learning Engineer),ЗП не указана,https://hh.ru/vacancy/73468878?query=data+engi...,hh.ru
2,Middle/Senior Data Engineer в Sber AI Lab,ЗП не указана,https://hh.ru/vacancy/73471713?query=data+engi...,hh.ru
3,NLP исследователь в группу AI в медицине Sber ...,ЗП не указана,https://hh.ru/vacancy/73469286?query=data+engi...,hh.ru
4,Data engineer,ЗП не указана,https://hh.ru/vacancy/72030869?query=data+engi...,hh.ru
...,...,...,...,...
495,Senior Expert (Engeeniring Data Managment) \ Г...,ЗП не указана,https://hh.ru/vacancy/71114250?query=data+engi...,hh.ru
496,Инженер (LabView),ЗП не указана,https://hh.ru/vacancy/69581088?query=data+engi...,hh.ru
497,Эксперт инженерных данных (EDM \ управление ин...,ЗП не указана,https://hh.ru/vacancy/71112507?query=data+engi...,hh.ru
498,Go Engineer (Golang Developer),ЗП не указана,https://hh.ru/vacancy/68581759?query=data+engi...,hh.ru


In [132]:
# сохраним vacancy_list в файл *.json

with open('output3.json', 'w', encoding = 'utf-8') as f:
    json.dump(vacancy_list, f)

In [133]:
# также сохраним DataFrame в файл *.csv

df.to_csv('output3.csv', encoding='utf-8-sig')